In [1]:
import json
import pandas
import pyarrow
from sqlalchemy import create_engine

# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [2]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [27]:
test_query = """
    select
        date_trunc(metrc_sales_receipts.sales_datetime, month) as date_type,
        metrc_sales_transactions.product_category_name,
        sum(metrc_sales_transactions.total_price) as total_gmv
    from
        companies
        inner join metrc_sales_receipts on metrc_sales_receipts.company_id = metrc_sales_receipts.company_id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'RA'
        and metrc_sales_receipts.sales_datetime >= '2020-01-01'
    group by
        1,
        2
    order by
        1 asc,
        2 asc
"""

incoming_transfer_packages_query = """
    select
        metrc_transfers.id as transfer_row_id,
        metrc_deliveries.id as delivery_row_id,
        metrc_transfer_packages.id as package_row_id,
        case
            when company_deliveries.delivery_type = 'INCOMING_UNKNOWN' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' then 'INCOMING_FROM_VENDOR'
            when company_deliveries.delivery_type = 'OUTGOING_UNKNOWN' then 'OUTGOING_TO_PAYOR'
            when company_deliveries.delivery_type = 'OUTGOING_TO_PAYOR' then 'OUTGOING_TO_PAYOR'
            else company_deliveries.delivery_type
        end as delivery_type,
        company_deliveries.license_number,
        metrc_transfers.manifest_number,
        metrc_transfers.created_date,
        metrc_deliveries.received_datetime,
        metrc_transfers.shipper_facility_license_number,
        metrc_transfers.shipper_facility_name,
        metrc_deliveries.recipient_facility_license_number,
        metrc_deliveries.recipient_facility_name,
        metrc_deliveries.shipment_type_name,
        metrc_deliveries.shipment_transaction_type,
        metrc_transfer_packages.package_id,
        metrc_transfer_packages.package_label,
        metrc_transfer_packages.type,
        metrc_transfer_packages.package_payload.shipmentpackagestate as shipment_package_state,
        metrc_transfer_packages.package_payload.istestingsample as is_testing_sample,
        metrc_transfer_packages.package_payload.istradesample as is_trade_sample,
        metrc_transfer_packages.product_category_name,
        metrc_transfer_packages.product_name,
        metrc_transfer_packages.lab_results_status as package_lab_results_status,
        metrc_transfer_packages.shipper_wholesale_price,
        metrc_transfer_packages.shipped_quantity,
        metrc_transfer_packages.shipped_unit_of_measure,
        metrc_transfer_packages.received_quantity,
        metrc_transfer_packages.received_unit_of_measure,
        metrc_transfer_packages.package_payload.itemunitweight as item_unit_weight,
        metrc_transfer_packages.package_payload.itemunitweightunitofmeasurename as item_unit_weight_unit_of_measure_name
    from
        metrc_transfers
        inner join company_deliveries on metrc_transfers.id = company_deliveries.transfer_row_id
        inner join companies on company_deliveries.company_id = companies.id
        inner join metrc_deliveries on metrc_transfers.id = metrc_deliveries.transfer_row_id
        inner join metrc_transfer_packages on metrc_deliveries.id = metrc_transfer_packages.delivery_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and (
            company_deliveries.delivery_type = 'INCOMING_FROM_VENDOR' or
            company_deliveries.delivery_type = 'INCOMING_UNKNOWN'
        )
        and metrc_transfers.created_date >= '2020-01-01'
    order by
        created_date desc
"""

sales_transactions_query = """
    select
        metrc_sales_receipts.id,
        metrc_sales_transactions.last_modified_at,
        metrc_sales_receipts.license_number,
        metrc_sales_receipts.receipt_id,
        metrc_sales_receipts.receipt_number,
        metrc_sales_receipts.type as receipt_type,
        metrc_sales_receipts.sales_customer_type,
        metrc_sales_receipts.sales_datetime,
        metrc_sales_receipts.total_packages,
        metrc_sales_receipts.total_price,
        metrc_sales_transactions.type as tx_type,
        metrc_sales_transactions.package_id as tx_package_id,
        metrc_sales_transactions.package_label as tx_package_label,
        metrc_sales_transactions.product_name as tx_product_name,
        metrc_sales_transactions.product_category_name as tx_product_category_name,
        metrc_sales_transactions.unit_of_measure as tx_unit_of_measure,
        metrc_sales_transactions.quantity_sold as tx_quantity_sold,
        metrc_sales_transactions.total_price as tx_total_price
    from
        metrc_sales_receipts
        inner join companies on metrc_sales_receipts.company_id = companies.id
        inner join metrc_sales_transactions on metrc_sales_receipts.id = metrc_sales_transactions.receipt_row_id
    where
        True
        and companies.identifier = 'HPCC'
        and metrc_sales_receipts.sales_datetime >= '2021-01-01'
    order by
        metrc_sales_receipts.sales_datetime desc
"""

simple_query = """
    SELECT COUNT(*) FROM `metrc_api_keys` LIMIT 1000
"""
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=BIGQUERY_CREDENTIALS_PATH)
# test = pandas.read_sql_query(incoming_transfer_packages_query, engine)
test = pandas.read_sql_query(test_query, engine)

In [28]:
test

,date_type,product_category_name,total_gmv
0,2020-01-01 00:00:00+00:00,Capsule (weight - each),11218.04
1,2020-01-01 00:00:00+00:00,Edible (volume - each),10441.50
2,2020-01-01 00:00:00+00:00,Edible (weight - each),266150.16
3,2020-01-01 00:00:00+00:00,Extract (volume - each),172.96
4,2020-01-01 00:00:00+00:00,Extract (weight - each),271166.95
...,...,...,...
617,2021-10-01 00:00:00+00:00,Tincture (weight - each),3127.16
618,2021-10-01 00:00:00+00:00,Topical (volume - each),5779.35
619,2021-10-01 00:00:00+00:00,Topical (weight - each),4215.47
620,2021-10-01 00:00:00+00:00,Vape Cartridge (volume - each),257337.33


In [34]:
test[0:100].groupby(['date_type', 'product_category_name']).sum()

total_gmv
date_type                 product_category_name                    
2020-01-01 00:00:00+00:00 Capsule (weight - each)          11218.04
                          Edible (volume - each)           10441.50
                          Edible (weight - each)          266150.16
                          Extract (volume - each)            172.96
                          Extract (weight - each)         271166.95
...                                                             ...
2020-04-01 00:00:00+00:00 Topical (volume - each)          13534.42
                          Topical (weight - each)          23569.24
                          Vape Cartridge (volume - each)  980275.73
                          Vape Cartridge (weight - each)   46879.35
2020-05-01 00:00:00+00:00 Capsule (weight - each)           9940.30

[100 rows x 1 columns]

In [ ]:
test.groupby(['date_type', 'product_category_name'])['total_gmv'].sum().unstack().plot.bar(figsize=(16, 8), stacked=True)